# Surface Simplification Using Quadric Error Metrics

Implementation of the surface simplification algorithm described in the paper [Surface Simplification Using Quadric Error Metrics](https://www.cs.cmu.edu/~./garland/Papers/quadrics.pdf) by Michael Garland and Paul S. Heckbert.

### Algorithm Summary
1. Compute the quadric error matrix, $Q$, for each vertex.
2. Select all valid pairs.
3. Compute the optimal target vertex, $\bar{v}$ for each pair. $\bar{v}^T (Q_1 + Q_2) \bar{v}$ is the cost of this pair.
4. Find the pair with the lowest cost and collapse it.  This can be done with a heap to speed up the process.

### Setup
Enter the project root directory and run

`python3 -m venv venv`

`source venv/bin/activate`

`pip3 install -r requirements.txt`

Then run `jupyter notebook` and open `surface_simplification.ipynb`
or use vscode and select venv as the python interpreter.

In [137]:
import numpy as np
import plotly.graph_objects as go

# MESH_FNAME = "../assets/bunny_1k.obj"
MESH_FNAME = "../assets/cube.obj"
# MESH_FNAME = "../assets/Model1.obj"

### Load Starting Mesh

In [138]:
def parse_obj_file(obj_file):
    """
    Parses a .obj file and returns a list of vertices and a face_idicies
    :param obj_file: .obj file to parse
    :return: array of vertices, array of faces
    """
    vertices = []
    faces = []
    with open(obj_file, "r") as f:
        data = f.readlines()
        for line in data:
            tokens = line.split()
            if len(tokens) > 0:
                if tokens[0] == "v":
                    vertex = []
                    vertex.append(float(tokens[1]))
                    vertex.append(float(tokens[2]))
                    vertex.append(float(tokens[3]))
                    vertices.append(vertex)
                elif tokens[0] == "f":
                    vertex_idxs = []
                    for token_idx in range(1, 4):
                        vertex_idx = int(tokens[token_idx].split("/")[0])
                        vertex_idxs.append(vertex_idx)
                    faces.append(vertex_idxs)
                else:
                    continue

    vertices = np.array(vertices)
    faces = np.array(faces)
    faces = faces - 1  # Convert from 1 index to 0 index
    return vertices, faces


def visualize_mesh(vertices, faces):
    """
    Visualizes a mesh using plotly
    :param vertices: array of vertices
    :param faces: array of faces
    """
    mesh = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color="lightpink",
        opacity=0.50,
    )

    scatter = go.Scatter3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        mode="markers",
        marker=dict(size=2, color="blue"),
    )
    camera = dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
    )

    fig = go.Figure(data=[mesh, scatter])
    fig.update_layout(scene_camera=camera, title="Original Plot")
    fig.show()

In [139]:
vertices, faces = parse_obj_file(MESH_FNAME)
visualize_mesh(vertices, faces)

### Compute Quadric Error Matrices

In [140]:
def compute_plane(p1, p2, p3):
    """
    Computes the plane defined by three points
    :param p1: first point
    :param p2: second point
    :param p3: third point
    :return: np.array([[a, b, c, d]]).T
    """
    v1 = p2 - p1
    v2 = p3 - p1
    normal = np.cross(v1, v2)
    normal = normal / np.linalg.norm(normal)
    d = -np.dot(normal, p1)
    return np.concatenate((normal, np.array([d]))).reshape((4, 1))


def get_Q_matricies(vertices, faces):
    """
    Computes the Q matrix for each vertex in the mesh
    :param vertices: array of vertices
    :param faces: array of faces
    :return: array of Q matrix for each vertex
    """
    Q_matricies = np.zeros((vertices.shape[0], 4, 4))
    for face_idx in range(faces.shape[0]):
        face = faces[face_idx]
        p = compute_plane(vertices[face[0]], vertices[face[1]], vertices[face[2]])
        K_p = p @ p.T
        for vertex_idx in face:
            Q_matricies[vertex_idx] += K_p

    return Q_matricies

In [141]:
Q_matricies = get_Q_matricies(vertices, faces)
print("First Q Matrix:")
print(Q_matricies[0])

First Q Matrix:
[[2. 0. 0. 2.]
 [0. 2. 0. 2.]
 [0. 0. 2. 2.]
 [2. 2. 2. 6.]]


### Select Valid Pairs

In [142]:
def get_pairs(faces, t):
    """
    Computes the pairs of vertices that are connected by an edge
    :param faces: array of faces
    :param t: threshold for distance between vertices.
    :return: array of vertex pairs

    Note: Set the threshold to 0 for large models.  We currently
    use a O(n^2) algorithm to compute the pairs.  This could probably be faster
    using KD trees or something similar.
    """
    pairs = set()

    # Add edges
    for face_idx in range(faces.shape[0]):
        face = faces[face_idx]
        for combo in [(0, 1), (1, 2), (2, 0)]:
            v1 = face[combo[0]]
            v2 = face[combo[1]]
            pair = min(v1, v2), max(v1, v2)
            pairs.add(pair)

    # Add thresholded distances
    if t > 0:
        for vertex_idx in range(vertices.shape[0]):
            for neighbor_idx in range(vertices.shape[0]):
                dist = np.linalg.norm(vertices[vertex_idx] - vertices[neighbor_idx])
                if (vertex_idx != neighbor_idx) and (dist < t):
                    pair = min(vertex_idx, neighbor_idx), max(vertex_idx, neighbor_idx)
                    pairs.add(pair)

    return pairs

In [143]:
pairs = get_pairs(faces, 4)

### Compute Contraction Targets and Costs

In [144]:
def get_cost(v1, v2, Q1, Q2):
    """
    Computes the cost of contracting v1 and v2
    :param v1: first vertex
    :param v2: second vertex
    :param Q1: Q matrix for v1
    :param Q2: Q matrix for v2
    :return: v_bar, cost of contracting v1 and v2
    """
    Q_bar = Q1 + Q2
    working_Q_bar = Q_bar.copy()
    working_Q_bar[3, :] = 0
    working_Q_bar[3, 3] = 1
    if np.linalg.cond(working_Q_bar) < 1/np.finfo(float).eps:
        v_bar = np.linalg.inv(working_Q_bar) @ np.array([0, 0, 0, 1]).T
    else:
        print("Singular Q")
        # Find best v by checking endpoints and midpoint if
        # Q_bar is not invertible
        c1 = v1.T @ Q_bar @ v1
        c2 = v2.T @ Q_bar @ v2
        v_mid = (v1 + v2) / 2
        c_bar = v_mid @ Q_bar @ v_mid

        if c_bar < c1 and c_bar < c2:
            v_bar = v_mid
        elif c1 < c2:
            v_bar = v1
        else:
            v_bar = v2

    return v_bar, v_bar.T @ Q_bar @ v_bar

for pair in pairs:
    v1 = vertices[pair[0]]
    v2 = vertices[pair[1]]
    Q1 = Q_matricies[pair[0]]
    Q2 = Q_matricies[pair[1]]
    v_bar, cost = get_cost(v1, v2, Q1, Q2)
    print(cost)

8.0
4.0
2.0
4.0
4.0
5.333333333333334
8.0
8.0
2.0
2.0
5.333333333333334
5.333333333333333
4.0
12.0
5.333333333333333
5.333333333333334
4.0
5.333333333333334
2.0
4.0
5.333333333333333
5.333333333333333
5.333333333333333
5.333333333333333
5.333333333333334
2.0
5.333333333333334
2.0
